#### Clean & Normalize the dataset (Guardian DRC)

This step will:
- Coalesce text columns,
- filter out very short pieces,
- parse dates, and export a tidy table for downstream NLP.


#### Import files and paths

In [13]:
import os, pandas as pd, re
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

RAW = Path("../data/raw/guardian_drc_2000_2025.csv")
OUT = Path("../data/interim/guardian_clean.parquet")

assert RAW.exists(), f"Missing raw file: {RAW} — put the CSV there first."
OUT.parent.mkdir(parents=True, exist_ok=True)

#### Load the data

In [3]:
df = pd.read_csv(RAW)
print("Raw shape:", df.shape) # Shows the columns and rows
df.head(5) # First 5 columns

Raw shape: (11497, 15)


,pub_date,section_name,headline,standfirst,trailText,bodyText,wordcount,byline,web_url,api_url,is_live,month_start,month_end,page_index,domain
0,2000-01-08T01:33:31Z,Books,Diary: We've been to the Dome and we're believers,NaN,<p>Disaster watch | More trouble with cyclists...,"I took my family to the Dome this week (yes, w...",1201,Simon Hoggart,https://www.theguardian.com/books/2000/jan/08/...,https://content.guardianapis.com/books/2000/ja...,NaN,2000-01-01,2000-01-31,1,theguardian.com
1,2000-01-14T03:47:30Z,Global,Would you really rather live in Ludwigshafen t...,"Well, according to a new survey, you would. O...",There's nothing playing at Ludwigshafen's Pfal...,There's nothing playing at Ludwigshafen's Pfal...,1082,NaN,https://www.theguardian.com/theguardian/2000/j...,https://content.guardianapis.com/theguardian/2...,NaN,2000-01-01,2000-01-31,1,theguardian.com
2,2000-01-15T00:36:48Z,World news,Belgium accused of killing African hero,Author urges parliamentary commission to quiz ...,Evidence of direct Belgian government complici...,Evidence of direct Belgian government complici...,665,Ian Black in Brussels,https://www.theguardian.com/world/2000/jan/15/...,https://content.guardianapis.com/world/2000/ja...,NaN,2000-01-01,2000-01-31,1,theguardian.com
3,2000-01-19T02:23:31Z,World news,Unita 'sanctions busters' named,NaN,The foreign office minister Peter Hain yesterd...,The foreign office minister Peter Hain yesterd...,454,"Ewen MacAskill, Diplomatic Editor",https://www.theguardian.com/world/2000/jan/19/...,https://content.guardianapis.com/world/2000/ja...,NaN,2000-01-01,2000-01-31,1,theguardian.com
4,2000-01-20T02:37:25Z,World news,Telegram exposes quest for personal riches in ...,NaN,Sordid details of the extent to which the Zimb...,Sordid details of the extent to which the Zimb...,544,Ewen MacAskill,https://www.theguardian.com/world/2000/jan/20/...,https://content.guardianapis.com/world/2000/ja...,NaN,2000-01-01,2000-01-31,1,theguardian.com


#### Schema sanity checks

In [4]:
REQUIRED = ["pub_date", "headline", "trailText", "bodyText", "web_url", "section_name"]
missing = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

df[REQUIRED].isna().sum()

pub_date          0
headline          0
trailText         9
bodyText        138
web_url           0
section_name      0
dtype: int64

We have 138 missing values in the bodyText. These can be dropped since they are not many compared to the actual corpus

#### Coalesce text (bodyText → trailText → headline)

Selecting the most complete or useful text field from a set of options, in a specific order of preference. This means extracting a meaningful piece of text from an article record in the following fields:
- bodyText – the full article text (most detailed)
- trailText – a short preview/snippet
- headline – the article title

In [7]:
# Define a function to coalesce the text

def coalesce_text(row, order=("bodyText", "trailText", "headline")):
    for c in order:
        val = row.get(c)
        if isinstance(val, str) and val.strip():
            return val
    return None

df["text"] = df.apply(coalesce_text, axis=1)
print("Rows with text:", df["text"].notna().sum(), " / ", len(df))
df[["headline","trailText","bodyText","text"]].head(5)


Rows with text: 11497  /  11497


,headline,trailText,bodyText,text
0,Diary: We've been to the Dome and we're believers,<p>Disaster watch | More trouble with cyclists...,"I took my family to the Dome this week (yes, w...","I took my family to the Dome this week (yes, w..."
1,Would you really rather live in Ludwigshafen t...,There's nothing playing at Ludwigshafen's Pfal...,There's nothing playing at Ludwigshafen's Pfal...,There's nothing playing at Ludwigshafen's Pfal...
2,Belgium accused of killing African hero,Evidence of direct Belgian government complici...,Evidence of direct Belgian government complici...,Evidence of direct Belgian government complici...
3,Unita 'sanctions busters' named,The foreign office minister Peter Hain yesterd...,The foreign office minister Peter Hain yesterd...,The foreign office minister Peter Hain yesterd...
4,Telegram exposes quest for personal riches in ...,Sordid details of the extent to which the Zimb...,Sordid details of the extent to which the Zimb...,Sordid details of the extent to which the Zimb...


#### Clean White spaces to filter by length

In [8]:
MIN_CHARS = 200

# normalize whitespace
df["text"] = (
    df["text"]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df["n_chars"] = df["text"].str.len()
before = len(df)
df = df[df["n_chars"] >= MIN_CHARS].copy()
after = len(df)
print(f"Filtered by length >= {MIN_CHARS}: {before} -> {after}")

Filtered by length >= 200: 11497 -> 11385


#### Parse dates and remove invalid ones

In [9]:
df["pub_date"] = pd.to_datetime(df["pub_date"], errors="coerce")
bad_dates = df["pub_date"].isna().sum()
if bad_dates:
    print("Dropping rows with invalid pub_date:", bad_dates)

df = df.dropna(subset=["pub_date"]).copy()
df["year"] = df["pub_date"].dt.year
df["month"] = df["pub_date"].dt.to_period("M").astype(str)

df[["pub_date","year","month","section_name","web_url","text"]].head(5)

/var/folders/z3/1tsrg0zj0qv7w9qv90t199n40000gn/T/ipykernel_54663/3775605852.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["pub_date"].dt.to_period("M").astype(str)


,pub_date,year,month,section_name,web_url,text
0,2000-01-08 01:33:31+00:00,2000,2000-01,Books,https://www.theguardian.com/books/2000/jan/08/...,"I took my family to the Dome this week (yes, w..."
1,2000-01-14 03:47:30+00:00,2000,2000-01,Global,https://www.theguardian.com/theguardian/2000/j...,There's nothing playing at Ludwigshafen's Pfal...
2,2000-01-15 00:36:48+00:00,2000,2000-01,World news,https://www.theguardian.com/world/2000/jan/15/...,Evidence of direct Belgian government complici...
3,2000-01-19 02:23:31+00:00,2000,2000-01,World news,https://www.theguardian.com/world/2000/jan/19/...,The foreign office minister Peter Hain yesterd...
4,2000-01-20 02:37:25+00:00,2000,2000-01,World news,https://www.theguardian.com/world/2000/jan/20/...,Sordid details of the extent to which the Zimb...


#### Check to remove Brazaville Congo if mentioned

In [10]:
import re
pat = re.compile(r"\b(Brazzaville|Republic of Congo|Congo-Brazzaville|RoC)\b", re.I)

def drop_brazzaville_rows(df_in: pd.DataFrame) -> pd.DataFrame:
    cols = [c for c in ["headline","trailText","bodyText","text"] if c in df_in.columns]
    m = pd.Series(False, index=df_in.index)
    for c in cols:
        m |= df_in[c].fillna("").str.contains(pat)
    return df_in.loc[~m].copy()

# Filter to check if there are nay changes in the dataset:
before = len(df)
df = drop_brazzaville_rows(df)
print(f"After excluding Brazzaville/RoC: {before} -> {len(df)}")

/var/folders/z3/1tsrg0zj0qv7w9qv90t199n40000gn/T/ipykernel_54663/1020320861.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m |= df_in[c].fillna("").str.contains(pat)


After excluding Brazzaville/RoC: 11385 -> 8745


#### Save to Parquet


In [11]:
clean = df[["pub_date","year","month","web_url","section_name","text"]].reset_index(drop=True)
clean.to_parquet(OUT, index=False)
print("Saved ->", OUT, "| rows:", len(clean))
clean.head(5)

Saved -> ../data/interim/guardian_clean.parquet | rows: 8745


,pub_date,year,month,web_url,section_name,text
0,2000-01-08 01:33:31+00:00,2000,2000-01,https://www.theguardian.com/books/2000/jan/08/...,Books,"I took my family to the Dome this week (yes, w..."
1,2000-01-15 00:36:48+00:00,2000,2000-01,https://www.theguardian.com/world/2000/jan/15/...,World news,Evidence of direct Belgian government complici...
2,2000-01-20 02:37:25+00:00,2000,2000-01,https://www.theguardian.com/world/2000/jan/20/...,World news,Sordid details of the extent to which the Zimb...
3,2000-01-20 17:58:25+00:00,2000,2000-01,https://www.theguardian.com/politics/2000/jan/...,Politics,"Tony Blair has overuled the foreign secretary,..."
4,2000-01-21 17:35:59+00:00,2000,2000-01,https://www.theguardian.com/world/2000/jan/21/...,World news,The government effectively abandoned any prete...


#### Check Final distribution

In [15]:
print("Date range:", clean["pub_date"].min(), "→", clean["pub_date"].max())
print("Sections (top 10):")
clean["section_name"].value_counts().head(20)

Date range: 2000-01-08 01:33:31+00:00 → 2025-10-29 05:00:35+00:00
Sections (top 10):


section_name
World news                2041
Global development        1163
Football                   770
Opinion                    694
Music                      345
US news                    318
Environment                290
Society                    230
Books                      211
UK news                    208
Business                   194
Politics                   187
Art and design             145
Sport                      136
Working in development     128
Australia news             127
Life and style             120
News                       119
Film                       118
Science                    114
Name: count, dtype: int64